# 🕵️ PawCore Intelligence Mystery Setup

**Complete setup for solving PawCore's Q4 revenue mystery using Snowflake Intelligence**

⚠️ **Run cells in order** - Each cell builds on the previous ones

## The Mystery
PawCore's Q4 revenue dropped in EMEA for SmartCollar products. Returns surged due to faulty batteries from Lot 341, but were under-reported by an overwhelmed support analyst. Use Snowflake Intelligence to solve this mystery end-to-end.

## What You'll Learn
- 🔍 **Semantic Views + Cortex Analyst**
- 🤖 **AI in SQL** (AI_EXTRACT, AI_CLASSIFY, AI_COMPLETE)
- 📄 **Cortex Search** for unstructured data
- 🚀 **Intelligent Agents** with custom tools
- 📧 **Automated executive reporting**


In [ ]:
-- 1️⃣ Environment Setup
USE ROLE ACCOUNTADMIN;

-- Create infrastructure
CREATE WAREHOUSE IF NOT EXISTS PAWCORE_INTELLIGENCE_WH
  WAREHOUSE_SIZE = 'XSMALL'
  AUTO_SUSPEND = 60
  AUTO_RESUME = TRUE;

CREATE DATABASE IF NOT EXISTS PAWCORE_INTELLIGENCE_DEMO;
CREATE SCHEMA IF NOT EXISTS PAWCORE_INTELLIGENCE_DEMO.BUSINESS_DATA;
CREATE SCHEMA IF NOT EXISTS PAWCORE_INTELLIGENCE_DEMO.DOCUMENTS;
CREATE SCHEMA IF NOT EXISTS PAWCORE_INTELLIGENCE_DEMO.AGENTS;

-- Set context
USE DATABASE PAWCORE_INTELLIGENCE_DEMO;
USE SCHEMA BUSINESS_DATA;
USE WAREHOUSE PAWCORE_INTELLIGENCE_WH;

SELECT '✅ Environment ready!' AS status;


In [ ]:
-- 2️⃣ Create Stages and File Formats
CREATE FILE FORMAT IF NOT EXISTS CSV_FORMAT
  TYPE = CSV
  FIELD_OPTIONALLY_ENCLOSED_BY = '"'
  SKIP_HEADER = 1
  NULL_IF = ('', 'NULL')
  TRIM_SPACE = TRUE;

CREATE STAGE IF NOT EXISTS INTERNAL_DATA_STAGE FILE_FORMAT = CSV_FORMAT;
CREATE STAGE IF NOT EXISTS DOCUMENT_STAGE;
CREATE STAGE IF NOT EXISTS AUDIO_STAGE;

-- Enable directory tables for file listing
ALTER STAGE INTERNAL_DATA_STAGE SET DIRECTORY = (ENABLE = TRUE);
ALTER STAGE DOCUMENT_STAGE SET DIRECTORY = (ENABLE = TRUE);
ALTER STAGE AUDIO_STAGE SET DIRECTORY = (ENABLE = TRUE);

SELECT '✅ Stages created!' AS status;


## ⬆️ **UPLOAD FILES NOW**

**Before continuing, upload these files via Snowflake UI:**

### 📁 To INTERNAL_DATA_STAGE:
- `pawcore_sales.csv`
- `returns.csv` ⭐ **Critical for mystery**
- `hr_resumes.csv`
- `device_sales_by_region.csv`
- `pet_owners.csv`
- `vet_feedback.csv`
- `fitbit_inventory.csv`
- `email_campaigns.csv`
- `enhanced_sales_data.csv`
- `social_media_posts.csv`
- `support_tickets.csv`
- `warranty_costs.csv`

### 📄 To DOCUMENT_STAGE:
- `customer_reviews.csv` ⭐ **Critical for mystery**
- `pawcore_slack.csv` ⭐ **Critical for mystery**
- `Q4_2024_PawCore_Financial_Report.md`
- `Sales_Performance_Q4_2024.md`
- `Quarterly_Sales_Speech_PawCore.md`

### 🎵 To AUDIO_STAGE:
- `PawCore Quarterly Call.mp3`

**📍 How to upload:** Go to Snowflake UI → Data → Databases → PAWCORE_INTELLIGENCE_DEMO → BUSINESS_DATA → click on stage name → "Upload Files"


In [ ]:
-- 3️⃣ Create All Tables

-- Core sales and business tables
CREATE TABLE IF NOT EXISTS PAWCORE_SALES (
  DATE DATE,
  REGION STRING,
  PRODUCT STRING,
  FORECAST_SALES NUMBER(10,2),
  ACTUAL_SALES NUMBER(10,2),
  VARIANCE NUMBER(10,2),
  PCT_OF_FORECAST NUMBER(5,2),
  INVENTORY_UNITS_AVAILABLE INTEGER,
  MARKETING_ENGAGEMENT_SCORE INTEGER
);

-- 🚨 Critical mystery table
CREATE TABLE IF NOT EXISTS RETURNS (
  RETURN_ID STRING,
  DATE DATE,
  REGION STRING,
  PRODUCT STRING,
  LOT_ID STRING,
  REASON STRING,
  QTY NUMBER,
  UNIT_COST_EUR NUMBER(10,2),
  TOTAL_COST_EUR NUMBER(10,2)
);

-- HR table for hiring bonus
CREATE TABLE IF NOT EXISTS HR_RESUMES (
  CANDIDATE_NAME STRING,
  RESUME_TEXT TEXT,
  APPLICATION_DATE DATE,
  POSITION_APPLIED STRING
);

-- Supporting business tables
CREATE TABLE IF NOT EXISTS PET_OWNERS (
  CUSTOMER_ID STRING,
  CUSTOMER_NAME STRING,
  PET_TYPE STRING,
  PET_NAME STRING,
  REGION STRING,
  SEGMENT STRING,
  JOIN_DATE DATE
);

CREATE TABLE IF NOT EXISTS DEVICE_SALES_BY_REGION (
  DATE DATE,
  REGION STRING,
  DEVICE_TYPE STRING,
  UNITS_SOLD INTEGER,
  REVENUE NUMBER(10,2),
  GROWTH_RATE NUMBER(5,2)
);

-- Unstructured data tables
CREATE TABLE IF NOT EXISTS SLACK_MESSAGES (
  MESSAGE_ID STRING,
  CHANNEL STRING,
  AUTHOR STRING,
  MESSAGE_TEXT STRING,
  TIMESTAMP TIMESTAMP,
  SENTIMENT STRING
);

CREATE TABLE IF NOT EXISTS CUSTOMER_REVIEWS (
  REVIEW_ID STRING,
  DATE DATE,
  PRODUCT STRING,
  CUSTOMER_NAME STRING,
  RATING INTEGER,
  REVIEW_TEXT STRING,
  SENTIMENT STRING,
  REVIEW_CATEGORY STRING,
  VERIFIED_PURCHASE STRING,
  HELPFUL_VOTES INTEGER,
  REVIEW_LENGTH INTEGER,
  REGION STRING,
  CUSTOMER_SEGMENT STRING
);

-- Mystery support tables
CREATE TABLE IF NOT EXISTS SUPPORT_TICKETS (
  TICKET_ID STRING,
  DATE DATE,
  REGION STRING,
  PRODUCT STRING,
  FIRMWARE_VERSION STRING,
  CATEGORY STRING,
  STATUS STRING
);

CREATE TABLE IF NOT EXISTS WARRANTY_COSTS (
  PERIOD STRING,
  REGION STRING,
  PRODUCT STRING,
  COST_EUR NUMBER(10,2)
);

SELECT '✅ All tables created!' AS status;


In [ ]:
-- 4️⃣ Load Critical Mystery Data

-- Core sales data
COPY INTO PAWCORE_SALES (
  DATE, REGION, PRODUCT, FORECAST_SALES, ACTUAL_SALES, VARIANCE,
  PCT_OF_FORECAST, INVENTORY_UNITS_AVAILABLE, MARKETING_ENGAGEMENT_SCORE
)
FROM @INTERNAL_DATA_STAGE/pawcore_sales.csv
FILE_FORMAT = CSV_FORMAT
ON_ERROR = 'CONTINUE';

-- 🚨 CRITICAL: Mystery returns data
COPY INTO RETURNS (
  RETURN_ID, DATE, REGION, PRODUCT, LOT_ID, REASON, QTY, UNIT_COST_EUR, TOTAL_COST_EUR
)
FROM @INTERNAL_DATA_STAGE/returns.csv
FILE_FORMAT = CSV_FORMAT
ON_ERROR = 'CONTINUE';

-- HR resumes for hiring solution
COPY INTO HR_RESUMES (
  CANDIDATE_NAME, RESUME_TEXT, APPLICATION_DATE, POSITION_APPLIED
)
FROM @INTERNAL_DATA_STAGE/hr_resumes.csv
FILE_FORMAT = CSV_FORMAT
ON_ERROR = 'CONTINUE';

-- Support evidence tables
COPY INTO SUPPORT_TICKETS (TICKET_ID, DATE, REGION, PRODUCT, FIRMWARE_VERSION, CATEGORY, STATUS)
FROM @INTERNAL_DATA_STAGE/support_tickets.csv
FILE_FORMAT = CSV_FORMAT
ON_ERROR = 'CONTINUE';

COPY INTO WARRANTY_COSTS (PERIOD, REGION, PRODUCT, COST_EUR)
FROM @INTERNAL_DATA_STAGE/warranty_costs.csv
FILE_FORMAT = CSV_FORMAT
ON_ERROR = 'CONTINUE';

SELECT '✅ Critical mystery data loaded!' AS status;


In [ ]:
-- 5️⃣ Load Unstructured Data with AI Enhancement

-- Load Slack messages via staging
CREATE OR REPLACE TABLE SLACK_MESSAGES_STG (URL STRING, CHANNEL STRING, TS STRING, TEXT STRING);
COPY INTO SLACK_MESSAGES_STG 
FROM @DOCUMENT_STAGE/pawcore_slack.csv 
FILE_FORMAT = CSV_FORMAT 
FORCE = TRUE 
ON_ERROR = 'CONTINUE';

-- Transform and enrich with sentiment
INSERT INTO SLACK_MESSAGES (MESSAGE_ID, CHANNEL, AUTHOR, MESSAGE_TEXT, TIMESTAMP, SENTIMENT)
SELECT 
    TS,
    CHANNEL,
    NULL,
    TEXT,
    TO_TIMESTAMP_NTZ(TRY_TO_NUMBER(SUBSTR(TS,1,10))),
    SNOWFLAKE.CORTEX.SENTIMENT(TEXT)  -- 🤖 AI sentiment analysis
FROM SLACK_MESSAGES_STG;

-- Load customer reviews
COPY INTO CUSTOMER_REVIEWS
FROM @DOCUMENT_STAGE/customer_reviews.csv
FILE_FORMAT = CSV_FORMAT
FORCE = TRUE
ON_ERROR = 'CONTINUE';

-- Load supporting customer data
CREATE OR REPLACE TABLE PET_OWNERS_STG (CUSTOMER_NAME STRING, PHONE STRING, EMAIL STRING);
COPY INTO PET_OWNERS_STG 
FROM @INTERNAL_DATA_STAGE/pet_owners.csv 
FILE_FORMAT = CSV_FORMAT 
ON_ERROR = 'CONTINUE';

INSERT INTO PET_OWNERS (CUSTOMER_ID, CUSTOMER_NAME, PET_TYPE, PET_NAME, REGION, SEGMENT, JOIN_DATE)
SELECT UUID_STRING(), CUSTOMER_NAME, NULL, NULL, NULL, NULL, CURRENT_DATE() 
FROM PET_OWNERS_STG;

SELECT '✅ Unstructured data loaded with AI enhancement!' AS status;


In [ ]:
-- 6️⃣ Verify Complete Setup
SELECT 
    table_name,
    row_count,
    CASE 
        WHEN table_name = 'RETURNS' AND row_count >= 9 THEN '🚨 Mystery data ready'
        WHEN table_name = 'CUSTOMER_REVIEWS' AND row_count >= 20 THEN '📝 Reviews loaded'
        WHEN table_name = 'SLACK_MESSAGES' AND row_count >= 15 THEN '💬 Slack + sentiment'
        WHEN table_name = 'HR_RESUMES' AND row_count >= 8 THEN '👥 Hiring candidates'
        WHEN row_count > 0 THEN '✅ Ready'
        ELSE '❌ Check data'
    END AS status
FROM (
    SELECT 'PAWCORE_SALES' AS table_name, COUNT(*) AS row_count FROM PAWCORE_SALES
    UNION ALL SELECT 'RETURNS', COUNT(*) FROM RETURNS
    UNION ALL SELECT 'HR_RESUMES', COUNT(*) FROM HR_RESUMES
    UNION ALL SELECT 'CUSTOMER_REVIEWS', COUNT(*) FROM CUSTOMER_REVIEWS
    UNION ALL SELECT 'SLACK_MESSAGES', COUNT(*) FROM SLACK_MESSAGES
    UNION ALL SELECT 'PET_OWNERS', COUNT(*) FROM PET_OWNERS
    UNION ALL SELECT 'SUPPORT_TICKETS', COUNT(*) FROM SUPPORT_TICKETS
    UNION ALL SELECT 'WARRANTY_COSTS', COUNT(*) FROM WARRANTY_COSTS
)
ORDER BY table_name;


In [ ]:
-- 7️⃣ Mystery Preview: Spot the Anomaly!
-- This query reveals the core mystery immediately

WITH weekly_returns AS (
    SELECT 
        DATE_TRUNC('WEEK', DATE) AS week_start,
        LOT_ID,
        REGION,
        SUM(QTY) AS returned_units,
        SUM(TOTAL_COST_EUR) AS total_cost
    FROM RETURNS
    WHERE PRODUCT = 'SmartCollar'
    GROUP BY 1, 2, 3
),
baseline AS (
    SELECT AVG(returned_units) AS avg_baseline
    FROM weekly_returns
    WHERE week_start < '2024-10-01'
)
SELECT 
    w.week_start,
    w.LOT_ID,
    w.REGION,
    w.returned_units,
    ROUND(w.total_cost, 0) AS cost_eur,
    ROUND(b.avg_baseline, 1) AS baseline,
    ROUND(w.returned_units / NULLIF(b.avg_baseline, 0), 1) AS spike_multiplier,
    CASE 
        WHEN w.returned_units >= 3 * b.avg_baseline THEN '🚨 ANOMALY DETECTED'
        WHEN w.returned_units >= 2 * b.avg_baseline THEN '⚠️ Elevated'
        ELSE 'Normal'
    END AS status
FROM weekly_returns w
CROSS JOIN baseline b
WHERE w.week_start >= '2024-10-01'
ORDER BY w.week_start, w.returned_units DESC;

-- 👆 You should see Lot 341 in EMEA with 3x+ spike!


## 🎉 **Setup Complete!**

**Your PawCore mystery environment is ready!**

### 🔍 **Key Evidence to Investigate:**
- **RETURNS**: Lot 341 anomalies in EMEA ⭐
- **CUSTOMER_REVIEWS**: Q4 SmartCollar complaints ⭐
- **SLACK_MESSAGES**: Support escalation patterns ⭐
- **SUPPORT_TICKETS**: Evidence of team overwhelm ⭐
- **WARRANTY_COSTS**: Financial impact escalation ⭐
- **HR_RESUMES**: Candidates for staffing solution

### 🚀 **Next Steps:**
1. 📖 Follow the **Master Guide**: `COMPLETE_MASTER_HOL_GUIDE.md`
2. 🕵️ Start with **Phase 1**: Detect the Gap using Cortex Analyst
3. 🔍 Create **Semantic Views** for business-friendly querying
4. 🤖 Use **AI in SQL** functions for deeper analysis
5. 📧 Build **Intelligent Agents** for automated reporting

### 💡 **Pro Tip:**
The mystery preview above should show **Lot 341** with a **3x+ spike** in returns. This is your starting clue!

**Ready to solve the mystery? Let's go! 🕵️‍♂️**
